In [ ]:
from object_3d import *
from camera_hand import *
from projection import *
import pygame as pg
import math



class SoftwareRender:
    def __init__(self):
        pg.init()
        self.RES = self.WIDTH, self.HEIGHT = 1600, 900
        self.H_WIDTH, self.H_HEIGHT = self.WIDTH // 2, self.HEIGHT // 2
        self.FPS = 120
        self.screen = pg.display.set_mode(self.RES)
        self.clock = pg.time.Clock()
        self.create_objects()

    def create_objects(self):
        self.camera = Camera(self, [-5, 6, -55])
        self.projection = Projection(self)
        self.object = self.get_object_from_file('build1.obj')
        self.object.rotate_y(-math.pi / 4)

    def get_object_from_file(self, filename):
        vertex, faces = [], []
        with open(filename) as f:
            for line in f:
                if line.startswith('v '):
                    vertex.append([float(i) for i in line.split()[1:]] + [1])
                elif line.startswith('f'):
                    faces_ = line.split()[1:]
                    faces.append([int(face_.split('/')[0]) - 1 for face_ in faces_])
        return Object3D(self, vertex, faces) 

    def draw(self):
        self.screen.fill(pg.Color('black'))
        self.object.draw()

    def run(self):
        while True:
            self.draw()
            self.camera.control()
            [exit() for i in pg.event.get() if i.type == pg.QUIT]
            pg.display.set_caption(str(self.clock.get_fps()))
    
            pg.display.flip()
            self.clock.tick(self.FPS)

if __name__ == '__main__':
    app = SoftwareRender()
    app.run()

In [ ]:
import numpy as np
import trimesh

# Load the 3D model
mesh = trimesh.load('InteriorTest.obj')

# Get the Trimesh object from the Scene object
trimesh_mesh = next(iter(mesh.geometry.values()))

# Get the vertices of the model
vertices = trimesh_mesh.vertices

# Calculate the center point
center_point = np.mean(vertices, axis=0)

# Print the center point coordinates
print("The center point of the model is:", center_point)


In [ ]:
import cv2
import numpy as np
import os
import glob

# Read the video
video_path = 'path/to/video.mp4'
cap = cv2.VideoCapture(video_path)

# Define the output directory to store the frames
output_dir = 'images/'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Extract frames from the video and save as image files
frame_num = 0
while(cap.isOpened()):
    ret, frame = cap.read()
    if ret == False:
        break
    frame_num += 1
    filename = output_dir + 'frame' + str(frame_num) + '.jpg'
    cv2.imwrite(filename, frame)

# Release the video capture object and close all windows
cap.release()
cv2.destroyAllWindows()

# Define the path to the image files
image_path = 'images/*.jpg'
images = glob.glob(image_path)

# Define the SIFT feature extractor and matcher
sift = cv2.xfeatures2d.SIFT_create()
matcher = cv2.FlannBasedMatcher({'algorithm': 0, 'trees': 5}, {})

# Extract SIFT features from the images
keypoints = []
descriptors = []
for image in images:
    img = cv2.imread(image, 0)
    kp, des = sift.detectAndCompute(img, None)
    keypoints.append(kp)
    descriptors.append(des)

# Match the SIFT features and estimate camera poses
matches = []
for i in range(len(descriptors) - 1):
    matches.append(matcher.knnMatch(descriptors[i], descriptors[i+1], k=2))

good_matches = []
for m, n in matches:
    if m.distance < 0.7 * n.distance:
        good_matches.append(m)

src_pts = []
dst_pts = []
for i in range(len(good_matches)):
    src_pts.append(keypoints[0][good_matches[i].queryIdx].pt)
    dst_pts.append(keypoints[1][good_matches[i].trainIdx].pt)

src_pts = np.float32(src_pts).reshape(-1, 1, 2)
dst_pts = np.float32(dst_pts).reshape(-1, 1, 2)

K = np.array([[1200, 0, 640], [0, 1200, 360], [0, 0, 1]])
E, mask = cv2.findEssentialMat(dst_pts, src_pts, K)

R1, R2, t = cv2.decomposeEssentialMat(E)

P1 = np.hstack((np.eye(3), np.zeros((3,1))))
P2 = np.hstack((R1, t))
P3 = np.hstack((R1, -t))
P4 = np.hstack((R2, t))
P5 = np.hstack((R2, -t))

points3D = cv2.triangulatePoints(P1, P2, src_pts, dst_pts)
points3D /= points3D[3]

# Generate a 3D model from the point cloud data
import pyvista as pv

cloud = pv.PolyData(points3D[:3].T)
cloud.plot()


In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# Load and preprocess the MNIST dataset
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
x_train = x_train.reshape(-1, 28, 28, 1).astype("float32") / 255.0
x_test = x_test.reshape(-1, 28, 28, 1).astype("float32") / 255.0

# Define the CNN architecture
model = keras.Sequential([
    layers.Conv2D(32, kernel_size=(3, 3), activation="relu", input_shape=(28, 28, 1)),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation="relu"),
    layers.Dense(10, activation="softmax")
])

# Compile the model
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

# Train the model
model.fit(x_train, y_train, batch_size=128, epochs=10, validation_split=0.1)

# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(x_test, y_test)
print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")


In [ ]:
model.save("mnist.h5")
print("Done")

In [ ]:
index = int(input("Enter a number (0 - 59999): "))
img = images[index]
plt.imshow(img.reshape(28, 28), cmap="Greys")